# Neeps - Hard

In [1]:
# Prerequesites
from pyhive import hive
%load_ext sql
%sql hive://cloudera@quickstart.cloudera:10000/sqlzoo
%config SqlMagic.displaylimit = 20

## 11.
**co.CHt is to be given all the teaching that co.ACg currently does. Identify those events which will clash.**

In [2]:
%%sql
WITH t AS (-- list all the events
  SELECT ut_event.id, ut_occurs.week, ut_event.dow,
    ut_teaches.staff, CAST(SPLIT(ut_event.tod, ':')[0] AS INT) AS begin, 
    CAST(SPLIT(ut_event.tod, ':')[0] AS INT) + ut_event.duration AS due
      FROM ut_event JOIN ut_teaches ON (ut_event.id=ut_teaches.event) JOIN
        ut_occurs ON (ut_event.id=ut_occurs.event)
)
SELECT DISTINCT a.id `co.CHt`, b.id `co.ACg`
  FROM (SELECT * FROM t WHERE staff='co.CHt') a JOIN
    (SELECT * FROM t WHERE staff='co.ACg') b ON (
        a.week=b.week AND a.dow=b.dow)
    WHERE
        (a.begin>=b.begin AND a.begin<b.due OR
         b.begin>=a.begin AND b.begin<a.due)

 * hive://cloudera@quickstart.cloudera:10000/sqlzoo
Done.


cht,acg
co12005.T03,co12005.T01
co12005.T03,co72013.L02
co12005.T03,co72013.T03


## 12.
**Produce a table showing the utilisation rate and the occupancy level for all rooms with a capacity more than 60.**

> I don't know how 'utilisation rate' and 'occupancy level' are defined.

In [9]:
%%sql
WITH hrs AS (
    -- theoretical full utilisation hours: 15 weeks * 5 days * 12 hours
    SELECT MAX(CAST(SPLIT(tod, ':')[0] AS INT) + duration) - 
             MIN(CAST(SPLIT(tod, ':')[0] AS INT)) hrs
        FROM ut_event
), t AS (
  SELECT SUM(hrs) tot_hrs FROM
    (SELECT ut_week.id, hrs.hrs
      FROM ut_week JOIN hrs) a
), s AS ( -- attended students per event
  SELECT room, SUM(sze) size, SUM(capacity) capacity FROM
    (SELECT ut_event.id evt, ut_occurs.week, ut_event.room, 
        ut_student.sze, ut_room.capacity FROM 
        ut_event JOIN ut_occurs ON (ut_event.id=ut_occurs.event) JOIN
        ut_attends ON (ut_event.id=ut_attends.event) JOIN
        ut_student ON (ut_student.id=ut_attends.student) JOIN
        ut_room ON (ut_event.room=ut_room.id)
     ) a
     GROUP BY room
)
SELECT ut_event.room, ROUND(SUM(duration)/t.tot_hrs, 2) util,
    ROUND(SUM(s.size)/SUM(s.capacity), 2) occup_level
    FROM ut_event JOIN ut_occurs ON (
        ut_event.id=ut_occurs.event) JOIN 
    ut_room ON (
        ut_event.room=ut_room.id) JOIN
    s ON (ut_event.room=s.room) JOIN
    t
        WHERE ut_room.capacity>60
        GROUP BY ut_event.room, t.tot_hrs

 * hive://cloudera@quickstart.cloudera:10000/sqlzoo
Done.


room,util,occup_level
cr.SMH,2.4,0.27


## 13.
**A one hour staff meeting is to be held between 09:00 and 17:00. Events which clash are to be cancelled. Identify the hour which will result in the least disruption.**

In [4]:
%%sql
WITH t AS ( -- all the events per hour
  SELECT * FROM
    (SELECT ut_occurs.event, ut_occurs.week, ut_event.dow, 
       CAST(SPLIT(ut_event.tod, ':')[0] AS INT) tod
       FROM ut_event JOIN ut_occurs ON (
           ut_event.id=ut_occurs.event)
       WHERE ut_event.duration=1
    UNION ALL
     SELECT ut_occurs.event, ut_occurs.week, ut_event.dow, 
       CAST(SPLIT(ut_event.tod, ':')[0] AS INT) + 1 AS tod
       FROM ut_event JOIN ut_occurs ON (
           ut_event.id=ut_occurs.event)
       WHERE ut_event.duration=2) AS a
)
SELECT ut_week.id week, 
  SUM(CASE WHEN dow='Monday' AND tod=9 THEN 1 ELSE 0 END) m09,
    SUM(CASE WHEN dow='Monday' AND tod=10 THEN 1 ELSE 0 END) m10,
    SUM(CASE WHEN dow='Monday' AND tod=11 THEN 1 ELSE 0 END) m11,
    SUM(CASE WHEN dow='Monday' AND tod=12 THEN 1 ELSE 0 END) m12,
    SUM(CASE WHEN dow='Monday' AND tod=13 THEN 1 ELSE 0 END) m13,
    SUM(CASE WHEN dow='Monday' AND tod=14 THEN 1 ELSE 0 END) m14,
    SUM(CASE WHEN dow='Monday' AND tod=15 THEN 1 ELSE 0 END) m15,
    SUM(CASE WHEN dow='Monday' AND tod=16 THEN 1 ELSE 0 END) m16,
    SUM(CASE WHEN dow='Monday' AND tod=17 THEN 1 ELSE 0 END) m17,    
    SUM(CASE WHEN dow='Tuesday' AND tod=9 THEN 1 ELSE 0 END) t09,
    SUM(CASE WHEN dow='Tuesday' AND tod=10 THEN 1 ELSE 0 END) t10,
    SUM(CASE WHEN dow='Tuesday' AND tod=11 THEN 1 ELSE 0 END) t11,
    SUM(CASE WHEN dow='Tuesday' AND tod=12 THEN 1 ELSE 0 END) t12,
    SUM(CASE WHEN dow='Tuesday' AND tod=13 THEN 1 ELSE 0 END) t13,
    SUM(CASE WHEN dow='Tuesday' AND tod=14 THEN 1 ELSE 0 END) t14,
    SUM(CASE WHEN dow='Tuesday' AND tod=15 THEN 1 ELSE 0 END) t15,
    SUM(CASE WHEN dow='Tuesday' AND tod=16 THEN 1 ELSE 0 END) t16,
    SUM(CASE WHEN dow='Tuesday' AND tod=17 THEN 1 ELSE 0 END) t17,    
    SUM(CASE WHEN dow='Wednesday' AND tod=9 THEN 1 ELSE 0 END) w09,
    SUM(CASE WHEN dow='Wednesday' AND tod=10 THEN 1 ELSE 0 END) w10,
    SUM(CASE WHEN dow='Wednesday' AND tod=11 THEN 1 ELSE 0 END) w11,
    SUM(CASE WHEN dow='Wednesday' AND tod=12 THEN 1 ELSE 0 END) w12,
    SUM(CASE WHEN dow='Wednesday' AND tod=13 THEN 1 ELSE 0 END) w13,
    SUM(CASE WHEN dow='Wednesday' AND tod=14 THEN 1 ELSE 0 END) w14,
    SUM(CASE WHEN dow='Wednesday' AND tod=15 THEN 1 ELSE 0 END) w15,
    SUM(CASE WHEN dow='Wednesday' AND tod=16 THEN 1 ELSE 0 END) w16,
    SUM(CASE WHEN dow='Wednesday' AND tod=17 THEN 1 ELSE 0 END) w17,    
    SUM(CASE WHEN dow='Thursday' AND tod=9 THEN 1 ELSE 0 END) th09,
    SUM(CASE WHEN dow='Thursday' AND tod=10 THEN 1 ELSE 0 END) th10,
    SUM(CASE WHEN dow='Thursday' AND tod=11 THEN 1 ELSE 0 END) th11,
    SUM(CASE WHEN dow='Thursday' AND tod=12 THEN 1 ELSE 0 END) th12,
    SUM(CASE WHEN dow='Thursday' AND tod=13 THEN 1 ELSE 0 END) th13,
    SUM(CASE WHEN dow='Thursday' AND tod=14 THEN 1 ELSE 0 END) th14,
    SUM(CASE WHEN dow='Thursday' AND tod=15 THEN 1 ELSE 0 END) th15,
    SUM(CASE WHEN dow='Thursday' AND tod=16 THEN 1 ELSE 0 END) th16,
    SUM(CASE WHEN dow='Thursday' AND tod=17 THEN 1 ELSE 0 END) th17,    
    SUM(CASE WHEN dow='Friday' AND tod=9 THEN 1 ELSE 0 END) f09,
    SUM(CASE WHEN dow='Friday' AND tod=10 THEN 1 ELSE 0 END) f10,
    SUM(CASE WHEN dow='Friday' AND tod=11 THEN 1 ELSE 0 END) f11,
    SUM(CASE WHEN dow='Friday' AND tod=12 THEN 1 ELSE 0 END) f12,
    SUM(CASE WHEN dow='Friday' AND tod=13 THEN 1 ELSE 0 END) f13,
    SUM(CASE WHEN dow='Friday' AND tod=14 THEN 1 ELSE 0 END) f14,
    SUM(CASE WHEN dow='Friday' AND tod=15 THEN 1 ELSE 0 END) f15,
    SUM(CASE WHEN dow='Friday' AND tod=16 THEN 1 ELSE 0 END) f16,
    SUM(CASE WHEN dow='Friday' AND tod=17 THEN 1 ELSE 0 END) f17
    FROM t RIGHT JOIN ut_week ON (ut_week.id=t.week)
    GROUP BY ut_week.id

 * hive://cloudera@quickstart.cloudera:10000/sqlzoo
Done.


week,m09,m10,m11,m12,m13,m14,m15,m16,m17,t09,t10,t11,t12,t13,t14,t15,t16,t17,w09,w10,w11,w12,w13,w14,w15,w16,w17,th09,th10,th11,th12,th13,th14,th15,th16,th17,f09,f10,f11,f12,f13,f14,f15,f16,f17
01,2,6,2,14,4,10,6,16,4,4,10,18,14,4,12,10,12,2,10,10,10,12,8,6,8,8,4,2,14,10,12,8,14,10,14,10,8,6,10,8,6,6,4,14,2
02,2,6,2,14,4,10,6,16,4,4,10,18,14,4,12,10,12,2,10,10,10,12,8,6,8,8,4,2,14,10,12,8,14,10,14,10,8,6,10,8,6,6,4,14,2
03,2,6,2,14,4,10,6,16,4,4,10,18,14,4,12,10,12,2,10,10,10,12,8,6,8,8,4,2,14,10,12,8,14,10,14,10,8,6,10,8,6,6,4,14,2
04,2,6,2,14,4,10,6,16,4,4,10,18,14,4,12,10,12,2,10,10,10,12,8,6,8,8,4,2,14,10,12,8,14,10,14,10,8,6,10,8,4,6,4,14,2
05,2,6,2,14,4,10,6,16,4,4,10,18,14,4,12,10,12,2,10,10,10,12,8,6,8,8,4,2,14,10,12,8,14,10,14,10,8,6,10,8,4,6,4,12,2
06,2,6,2,14,4,10,6,16,4,4,10,18,14,4,12,10,12,2,10,10,10,12,8,6,8,8,4,2,14,10,12,8,14,10,14,10,8,6,10,8,4,6,4,12,2
07,2,6,2,14,4,10,6,16,4,4,10,18,14,4,12,10,12,2,10,10,10,12,8,6,8,8,4,2,14,10,12,8,14,10,14,10,8,6,10,8,4,6,4,12,2
08,2,6,2,14,4,10,6,16,4,4,10,18,14,4,12,10,12,2,10,10,9,12,8,6,8,8,4,2,14,10,12,8,14,10,14,10,8,6,10,8,4,6,4,12,2
09,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10,2,6,2,14,4,10,6,16,4,4,10,18,14,4,12,10,12,2,10,10,10,12,8,6,8,8,4,2,14,10,12,8,14,10,12,10,8,6,10,8,4,6,6,12,2


In [5]:
%%sql
WITH t AS (
-- You may find that week '09', '14', '15' are totally vacant
-- or you can choose Mon 9am/12pm, Tue 5pm, Thu 9am, Fri 5pm 
    -- all the events per hour
  SELECT * FROM
    (SELECT ut_occurs.event, ut_occurs.week, ut_event.dow, 
       CAST(SPLIT(ut_event.tod, ':')[0] AS INT) tod
       FROM ut_event JOIN ut_occurs ON (
           ut_event.id=ut_occurs.event)
       WHERE ut_event.duration=1
    UNION ALL
     SELECT ut_occurs.event, ut_occurs.week, ut_event.dow, 
       CAST(SPLIT(ut_event.tod, ':')[0] AS INT) + 1 AS tod
       FROM ut_event JOIN ut_occurs ON (
           ut_event.id=ut_occurs.event) 
       WHERE ut_event.duration=2) AS a
)
SELECT tod, dow, COUNT(*) n
    FROM t
    WHERE tod BETWEEN 9 AND 17
    GROUP BY tod, dow
    ORDER BY n, tod, dow

 * hive://cloudera@quickstart.cloudera:10000/sqlzoo
Done.


tod,dow,n
9,Monday,24
9,Thursday,24
11,Monday,24
17,Friday,24
17,Tuesday,24
9,Tuesday,48
13,Monday,48
13,Tuesday,48
17,Monday,48
17,Wednesday,48


## 14.
**Find all clashes - include the events which clash and the staff, student or rooms that they have in common.**

In [6]:
%%sql
SELECT 'don\'t know how \'clash\' is defined' AS huh FROM ut_event LIMIT 1

 * hive://cloudera@quickstart.cloudera:10000/sqlzoo
Done.


huh
don't know how 'clash' is defined


## 15.
**Produce a timetable for a group of full time students for week 1**

In [7]:
%%sql
WITH t AS (-- vague requirements
    SELECT ut_event.id, student, dow, CAST(SPLIT(tod, ':')[0] AS INT) tod
      FROM ut_event JOIN ut_occurs ON (ut_event.id=ut_occurs.event)
        JOIN ut_attends ON (ut_attends.event=ut_event.id)
        WHERE ut_occurs.week='01' AND duration=1
    UNION ALL
    SELECT ut_event.id, student, dow, CAST(SPLIT(tod, ':')[0] AS INT) + 1 AS tod
      FROM ut_event JOIN ut_occurs ON (ut_event.id=ut_occurs.event)
        JOIN ut_attends ON (ut_attends.event=ut_event.id)
        WHERE ut_occurs.week='01' AND duration=2
)
SELECT student, tod,
  MAX(CASE WHEN dow='Monday' THEN id ELSE NULL END) mon,
    MAX(CASE WHEN dow='Tuesday' THEN id ELSE NULL END) tue,
    MAX(CASE WHEN dow='Wednesday' THEN id ELSE NULL END) wed,
    MAX(CASE WHEN dow='Thursday' THEN id ELSE NULL END) thu,
    MAX(CASE WHEN dow='Friday' THEN id ELSE NULL END) fri
    FROM t
    GROUP BY student, tod
    ORDER BY student, tod

 * hive://cloudera@quickstart.cloudera:10000/sqlzoo
Done.


student,tod,mon,tue,wed,thu,fri
co.12008.Ea,12,None,co12008.L01,None,None,None
co.12008.Ea,14,co12008.T01,None,None,None,None
co.12008.Eb,12,None,co12008.L01,None,None,None
co.12008.Eb,15,None,None,None,None,co12008.T02
co.12012.E,12,None,None,co12012.L01,None,None
co.12012.E,14,None,None,co12012.T01,None,None
co.22022.E,9,None,None,None,None,co22022.T01
co.22022.E,11,None,None,None,None,co22022.L01
co1.BAe,10,co12011.L01,None,co12004.T05,co12006.L02,None
co1.BAe,11,None,None,co12004.L01,None,None
